In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Clustering process for policies

## 1. Import the relevant packages

In [2]:
import pandas as pd
import random

from process import extract_items
import clusterize

2025-07-16 14:30:25.810540: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-16 14:30:25.837167: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-16 14:30:25.880101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-16 14:30:25.925494: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-16 14:30:25.925534: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-16 14:30:25.983738: I tensorflow/core/platform/cpu_feature_guard.cc:

## 2. Prepare data for clustering

In [141]:
domain = 'E'
input_path = f"../data/results_{domain}.csv"

In [142]:
# Import data
df = pd.read_csv(input_path)
df

,GEOGRAPHIC,ITEM,FACTOR,SECTOR,abstract,doi,openalex_id
0,Mashhad University of Medical Sciences,mass media,"{""awareness about the benefits of physical act...","[""E""]",Background and Aim: Health-promoting lifestyle...,https://doi.org/10.22038/jmrh.2014.2918,W2099730092
1,United States,exergames,"{""physical activity (PA) in the general popula...","[""E"", ""J""]",The number of informal caregivers for family m...,https://doi.org/10.48550/arxiv.1908.09984,W2970340995
2,Puget Sound,telecommunications,"{""trip generation rates"": {""CORRELATION"": ""dec...","[""E""]",Most trip generation models are insensitive to...,https://doi.org/10.3141/1682-09,W2129598736
3,US,activewear purchased to be worn as casualwear,"{""intent to purchase activewear"": {""CORRELATIO...","[""E""]","In recent years, activewear is no longer conta...",https://doi.org/10.1080/17543266.2018.1477995,W2805060567
4,Indonesia,"advanced, automated internet technology","{""robust connectivity"": {""CORRELATION"": ""incre...","[""E""]",Abstract The Indonesian government has a goal ...,https://doi.org/10.2118/209896-ms,W4290471442
...,...,...,...,...,...,...,...
6935,"Samarinda, East Kalimantan province, Indonesia",IoT-based air pollution monitoring system usin...,"{""air quality information accessibility"": {""CO...","[""E""]",The degradation of air quality in numerous Ind...,https://doi.org/10.51967/tanesa.v24i2.2946,W4399888694
6936,Democratic Republic of Congo (DRC),privately funded GSM network,"{""mobile telephone coverage"": {""CORRELATION"": ...","[""E""]",The Democratic Republic of Congo (DRC) faces p...,NaN,W3121765686
6937,Thailand,watching television and playing computer,"{""obesity"": {""CORRELATION"": ""increasing""}}","[""E""]","Objective: To determine the dietary pattern, ...",NaN,W2161121663
6938,Pakistan,social media,"{""biogas technology adoption"": {""CORRELATION"":...","[""E""]",Environmental degradation and rapid climate ch...,https://doi.org/10.3390/ijerph17072311,W3013389524


In [159]:
# Apply the extraction
extracted_items_df = extract_items(df)
extracted_items_df

"rainfall estimation"
"rainfall estimation"


,index,ITEM,FACTOR
0,0,mass media,awareness about the benefits of physical activity
1,1,exergames,physical activity (PA) in the general population
2,1,exergames,PA barriers that AD caregivers face
3,2,telecommunications,trip generation rates
4,3,activewear purchased to be worn as casualwear,intent to purchase activewear
...,...,...,...
10376,6935,IoT-based air pollution monitoring system usin...,air quality information accessibility
10377,6936,privately funded GSM network,mobile telephone coverage
10378,6937,watching television and playing computer,obesity
10379,6938,social media,biogas technology adoption


## 3. Cluster with HDBSCAN

### a. ITEMS

In [6]:
preprocessed_items = extracted_items_df[['index', 'ITEM']].drop_duplicates(subset='index')['ITEM']
preprocessed_items

0                                               mass media
1                                                exergames
3                                       telecommunications
4            activewear purchased to be worn as casualwear
6                  advanced, automated internet technology
                               ...                        
10376    IoT-based air pollution monitoring system usin...
10377                         privately funded GSM network
10378             watching television and playing computer
10379                                         social media
10380                                         social media
Name: ITEM, Length: 6890, dtype: object

In [7]:
corpus_list_items, corpus_embeddings_items = clusterize.get_embeddings(preprocessed_items)

In [72]:
params = {
    'umap_n_neighbors': 30,
    'umap_min_dist': 0.1,
    'umap_n_components': 30,
    'hdbscan_min_cluster_size': 70,
    'hdbscan_min_samples': 10,
    'hdbscan_cluster_selection_epsilon': 0.0,
    'hdbscan_max_cluster_size': None
}

hdbscan_model, cluster_assignment, clustered_sentences, noise, reduced_embeddings_items = clusterize.clusterize(
                            corpus_list_items, corpus_embeddings_items,
                            params,
                            print_clusters=False)

/home/thomas/.pyenv/versions/lewagon-decision-science/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Number of clusters found: 33


In [73]:
results = {
    'items_total': len(cluster_assignment),
    'items_noise': len(noise),
    'noise_pct': round(len(noise)/len(cluster_assignment) * 100, 2)
}

print(f'total number of items: {results['items_total']}, \nitems classified as noise: {results['items_noise']}, \npercentage of noise: {results['noise_pct']}')

total number of items: 6890, 
items classified as noise: 1542, 
percentage of noise: 22.38


In [10]:
cluster_summary_df = clusterize.get_cluster_summary(clustered_sentences)
cluster_summary_df

,Cluster Number,Number of Sentences,Sample Sentences
0,1,58,reduced social media use (SMU); social Interne...
1,2,23,non-use of the internet; using the internet; b...
2,3,40,information and communications technologies (I...
3,4,56,Internet-of-Things; IoT based smart homes; Int...
4,5,23,digital optimization; Digitalization; digitali...
5,6,27,five-feature website; Korean pop cultural cont...
6,7,53,Big Data; online cloud services; cloud computi...
7,8,91,digital inclusion of civil society; digital so...
8,9,68,HPC/ICT servers operation and cooling; energy ...
9,10,47,large data center; data center; data centers; ...


In [11]:
noise_df = clusterize.get_noise_sample(noise)
noise_df

,samples
0,TikTok's localised moderation
1,real-time smart charging and storage platform
2,mobile phone addiction
3,virtual reality (VR)
4,online social support
...,...
95,information and communications technology
96,domain-specific preprocessing for wastewater a...
97,internet shutdowns
98,demand response (DR)


In [12]:
clusterize.save_results(params, results, noise_df, cluster_summary_df, hdbscan_model, domain, 'items')

In [169]:
from sklearn.cluster import HDBSCAN

## Re-clsutering of subdivision of clusters
### List of clusters to subdivide
clusters_to_subdivide_with_params = {
    7: {'min_cluster_size': 15, 'min_samples': 5, 'cluster_selection_epsilon': 0.1},
    9: {'min_cluster_size': 10, 'min_samples': 5, 'cluster_selection_epsilon': 0.1},
    16: {'min_cluster_size': 15, 'min_samples': 5, 'cluster_selection_epsilon': 0.1},
    23: {'min_cluster_size': 15, 'min_samples': 5, 'cluster_selection_epsilon': 0.1},
    24: {'min_cluster_size': 15, 'min_samples': 5, 'cluster_selection_epsilon': 0.1},
    27: {'min_cluster_size': 17, 'min_samples': 5, 'cluster_selection_epsilon': 0.1},
    30: {'min_cluster_size': 17, 'min_samples': 5, 'cluster_selection_epsilon': 0.1}
}

# Subdivide selected clusters and create a new dataframe
new_cluster_df, new_noise, cluster_assignment_to_change_indices, new_cluster_assignment = clusterize.subdivide_clusters(
    clustered_sentences, cluster_assignment, reduced_embeddings_items, preprocessed_items, clusters_to_subdivide_with_params
)

# Display the new dataframe
new_cluster_df

no noise generated by cluster 7
reclustering of cluster 9 adding 39 items to noise
reclustering of cluster 16 adding 6 items to noise
no noise generated by cluster 23
reclustering of cluster 24 adding 7 items to noise
reclustering of cluster 27 adding 19 items to noise
reclustering of cluster 30 adding 2 items to noise


,cluster_id,sentences
0,0,"[social media usage, social media use, social ..."
1,1,"[internet usage, Internet use, internet use, i..."
2,2,[information and communication technologies (I...
3,3,"[Internet of Things utilization, technological..."
4,4,"[digitalisation of societies, technology and d..."
5,5,"[social media, social media, social media, web..."
6,6,"[smart city policies, smart city, Cloud Comput..."
7,8,"[Eco-design guidelines, strategies, and digita..."
8,10,"[ICT utilization program, information technolo..."
9,11,"[social media collaboration, face-to-face coll..."


In [175]:
print(f'{new_cluster_df.shape[0] - cluster_summary_df.shape[0] + len(clusters_to_subdivide_with_params.keys())} new clusters were created out of {len(clusters_to_subdivide_with_params.keys())}')
print(f'{len(new_noise)} items from the previous clusters were classified as noise')

14 new clusters were created out of 7
73 items from the previous clusters were classified as noise


In [81]:
new_noise[:10]

['online pizza ordering system',
 'cloud radio access network (C-RAN) architecture',
 'optimization problem to allocate radio resources',
 'flexible service level agreements in Green IT',
 'Cloud Radio Access Network (C-RAN) architecture',
 'green communication technologies',
 'radio access network (RAN) energy consumption',
 'Green procurement',
 'Reverse Logistics in Green Procurement',
 'Cloud Radio Access Network (C-RAN) architecture']

In [193]:
data = {'noise_samples': random.sample(new_noise, min(100, len(new_noise)))}

# Creating DataFrame
new_noise_df = pd.DataFrame(data)

new_noise_df.to_csv(f'../outputs/noise_sample_recluster_{domain}_items.csv', index=False)

new_noise_df

,noise_samples
0,power models for modern data centers
1,geo-distributed data centers (DCs)
2,geo-distributed data centers (DCs)
3,electricity mix
4,wireless technology in conjunction with a webs...
...,...
68,Internet data centers (IDCs) and the distribut...
69,online technologies
70,data centers (DCs)
71,Internet data centers (IDCs) and the distribut...


In [180]:
sentences_df = pd.DataFrame(preprocessed_items.copy())

doi_col = df[(df['FACTOR'] != "\"None\"") & (df['FACTOR'] != "\"rainfall estimation\"")]['doi']

final_df = clusterize.create_final_df(sentences_df, doi_col, cluster_assignment, new_cluster_assignment, cluster_assignment_to_change_indices)
final_df

,ITEM,cluster,cluster_2,doi
0,mass media,21,21,https://doi.org/10.22038/jmrh.2014.2918
1,exergames,17,17,https://doi.org/10.48550/arxiv.1908.09984
3,telecommunications,23,23_0,https://doi.org/10.1080/17543266.2018.1477995
4,activewear purchased to be worn as casualwear,26,26,https://doi.org/10.2118/209896-ms
6,"advanced, automated internet technology",27,-1,https://doi.org/10.2478/aup-2023-0012
...,...,...,...,...
10376,IoT-based air pollution monitoring system usin...,-1,-1,NaN
10377,privately funded GSM network,23,23_0,NaN
10378,watching television and playing computer,24,24_1,NaN
10379,social media,5,5,NaN


In [194]:
final_df.to_csv(f'../outputs/clusters_recluster_hdbscan_{domain}_items.csv', index=False)

In [186]:
recluster_params = pd.DataFrame.from_dict(clusters_to_subdivide_with_params, orient='index')
recluster_params.index.name = 'cluster_id'

recluster_params.to_csv(f'../outputs/params_recluster_{domain}_items.csv')

recluster_params

,min_cluster_size,min_samples,cluster_selection_epsilon
cluster_id,,,
7,15,5,0.1
9,10,5,0.1
16,15,5,0.1
23,15,5,0.1
24,15,5,0.1
27,17,5,0.1
30,17,5,0.1


### c. FACTORS

In [41]:
preprocessed_factors = extracted_items_df['FACTOR']
preprocessed_factors

0        awareness about the benefits of physical activity
1         physical activity (PA) in the general population
2                      PA barriers that AD caregivers face
3                                    trip generation rates
4                            intent to purchase activewear
                               ...                        
10376                air quality information accessibility
10377                            mobile telephone coverage
10378                                              obesity
10379                           biogas technology adoption
10380                                       social support
Name: FACTOR, Length: 10381, dtype: object

In [7]:
corpus_list_factors, corpus_embeddings_factors = clusterize.get_embeddings(preprocessed_factors)

In [ ]:
params = {
    'umap_n_neighbors': 30,
    'umap_min_dist': 0.1,
    'umap_n_components': 40,
    'hdbscan_min_cluster_size': 40,
    'hdbscan_min_samples': 5,
    'hdbscan_cluster_selection_epsilon': 0.0,
    'hdbscan_max_cluster_size': 370
}

hdbscan_model, cluster_assignment, clustered_sentences, noise, reduced_embeddings_factors = clusterize.clusterize(
                            corpus_list_factors, corpus_embeddings_factors,
                            params,
                            print_clusters=False)

/home/thomas/.pyenv/versions/lewagon-decision-science/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Number of clusters found: 74


In [94]:
results = {
    'factors_total': len(cluster_assignment),
    'factors_noise': len(noise),
    'noise_pct': round(len(noise)/len(cluster_assignment) * 100, 2)
}

print(f'total number of factors: {results['factors_total']}, \nfactors classified as noise: {results['factors_noise']}, \npercentage of noise: {results['noise_pct']}')

total number of factors: 6890, 
factors classified as noise: 1271, 
percentage of noise: 18.45


In [95]:
cluster_summary_df = clusterize.get_cluster_summary(clustered_sentences)
cluster_summary_df

,Cluster Number,Number of Sentences,Sample Sentences
0,1,12,bedtime; poor sleep quality; insomnia; sleep d...
1,2,18,energy consumption stabilization; overall ener...
2,3,25,power efficiency; energy comprehensive utiliza...
3,4,8,total energy footprint; carbon footprints; env...
4,5,13,transition to circular economy; circular econo...
...,...,...,...
69,70,18,knowledge access; learning; technical proficie...
70,71,67,information interaction; access to information...
71,72,22,user continuous intentions; behavioral intenti...
72,73,53,sustainable telephone and Internet connectivit...


In [97]:
noise_df = clusterize.get_noise_sample(noise)
noise_df

,samples
0,job satisfaction
1,evaporation estimation accuracy
2,geo-spatial coverage
3,reading performance
4,GHG emissions
...,...
95,EVs and their impacts on power systems
96,Internet addiction
97,health knowledge
98,power consumption


In [98]:
clusterize.save_results(params, results, noise_df, cluster_summary_df, hdbscan_model, domain, 'factors')